## Requeriments

In [ ]:
# Instal boto3 
%pip install boto3

# Install dotenv
%pip install python-dotenv

In [9]:
# Reload notebook to have saved changes of others modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from dotenv import load_dotenv
from pathlib import Path
from PIL import Image
import pandas as pd
import tarfile
import shutil
import boto3
import cv2
import os
load_dotenv()

from prepare_dataset import unzip_data

In [4]:
# Initiate AWS env variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
BUCKET_NAME = os.getenv('BUCKET_NAME')
BUCKET_PREFIX = os.getenv('BUCKET_PREFIX')

# Local paths
BASE_DIR= os.getenv('BASE_DIR')
PATH_TO_SAVE = os.getenv('PATH_TO_SAVE')
path_to_save = os.path.join(BASE_DIR, PATH_TO_SAVE)

zip_path = os.path.join(BASE_DIR, os.getenv('zip_path'))

# Other variables
download_s3 = False

In [33]:
# Import libraries
# Classic libraries
from dotenv import load_dotenv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import shutil
import os
load_dotenv()

# Custom functions
# from modules import plot_bounding_boxes
import prepare_dataset
from prepare_dataset import get_file_folders
from prepare_dataset import unzip_data
from prepare_dataset import download_files
from prepare_dataset import concatenate_csv
from prepare_dataset import check_bad_files
from prepare_dataset import plot_bounding_boxes
from prepare_dataset import mov_images
# AWS
import boto3

# Open CV
import cv2

In [ ]:
unzip_data(zip_path,
               path_to_save,
               zip_file="SKU110K_fixed.tar.gz")

In [2]:
# Initiate AWS env variables
ACCESS_KEY = os.getenv('ACCESS_KEY')
SECRET_KEY = os.getenv('SECRET_KEY')
BUCKET_NAME = os.getenv('BUCKET_NAME')
BUCKET_PREFIX = os.getenv('BUCKET_PREFIX')

# Local paths
BASE_DIR= os.getenv('BASE_DIR')
PATH_TO_SAVE = os.getenv('PATH_TO_SAVE')
path_to_save = os.path.join(BASE_DIR, PATH_TO_SAVE)

# csv files directions
csv_paths  = os.path.join(BASE_DIR, os.getenv('csv_paths'))
txt_file   = os.path.join(csv_paths, os.getenv('txt_file'))
test_file  = os.path.join(csv_paths, os.getenv('test_file')) 
val_file   = os.path.join(csv_paths, os.getenv('val_file'))      
train_file = os.path.join(csv_paths, os.getenv('train_file')) 

# Important paths
final_images_dir = os.path.join(BASE_DIR, os.getenv('final_images_dir'))
images_dir = os.path.join(BASE_DIR, os.getenv('images_dir'))
split_dir = os.path.join(BASE_DIR, os.getenv('split_dir'))

## Download information from AWS S3

"wget http://trax-geometry.s3.amazonaws.com/cvpr_challenge/SKU110K_fixed.tar.gz"

## Evaluation EDA

##### csv
- Validad que no existan valores nulos.
- Validar valores enteros en coordendas.
- Validar que sean positivos.
- Validar que los xs y ys no sean iguales.
- Validar superposición de bounding boxes

##### Imagenes & csv
- Validar imagenes que no tengan bounding boxes con referencia con csv.

##### Imagenes
- Evaluar todas las imágenes con el fin de encontrar imágenes corruptas <br> o con formato no valido.


- pendientes: Validar que los bounding boxes esten en el rango correcto.

In [4]:
# Get concatenated dataframes
df = concatenate_csv(txt_file,
     test_file,
     train_file,
     val_file)

In [5]:
# Get the amount of classes
df['class'].value_counts()

object    1730996
Name: class, dtype: int64

In [5]:
# Check data info and check that coordinates and image size are int types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1730996 entries, 0 to 1730995
Data columns (total 9 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   image_name    object
 1   x1            int64 
 2   y1            int64 
 3   x2            int64 
 4   y2            int64 
 5   class         object
 6   image_width   int64 
 7   image_height  int64 
 8   set           object
dtypes: int64(6), object(3)
memory usage: 118.9+ MB


In [6]:
# Check stats about the info
df.describe().round(2)

,x1,y1,x2,y2,image_width,image_height
count,1730996.00,1730996.00,1730996.00,1730996.00,1730996.00,1730996.00
mean,1173.73,1531.98,1294.15,1721.80,2452.50,3220.00
std,709.15,824.72,711.42,813.46,425.52,677.16
min,0.00,0.00,21.00,21.00,480.00,640.00
25%,595.00,898.00,716.00,1098.00,2336.00,2560.00
50%,1153.00,1498.00,1270.00,1675.00,2448.00,3264.00
75%,1702.00,2106.00,1820.00,2286.00,2448.00,3264.00
max,4129.00,4730.00,4323.00,5168.00,4320.00,5312.00


In [7]:
# Check null values in dataframe
df.isnull().value_counts()

image_name  x1     y1     x2     y2     class  image_width  image_height  set  
False       False  False  False  False  False  False        False         False    1730996
dtype: int64

In [108]:
# Check for negative values
numeric_df = df.drop(['image_name', 'class', 'set'], axis=1)

negatives = numeric_df.apply(lambda x : x < 0)
negatives = numeric_df[negatives.any(1)]

print(f"There is {len(negatives)} negative values")

There is 0 negative values


In [106]:
# Check we have coordinates that creates an area
df["area"] = (df["x2"] - df["x1"]) * (df["y2"] - df["y1"])

df_without_area = df[df["area"] == 0]

print(f"{len(df_without_area)} coordinates don't create an area")

0 coordinates don't create an area


In [105]:
# Check every image has its own bounding boxes
# List every filename in the "data/images" folder
images_list = set(os.listdir(images_dir))

# List unique image names from csv
images_csv = set(df["image_name"])

# Find differences between sets
diff = (images_list - images_csv)

print(f"{len(diff)} images doesn't have bounding box")

0 images doesn't have bounding box


In [104]:
# Evaluate corrupt images
bad_files = check_bad_files(images_dir)

print(f"There are {len(bad_files)} corrupt images")

There are 0 corrupt images


## 